In [1]:
#pip install ctgan

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import xgboost as xgb
from xgboost import XGBRegressor
#from ctgan import CTGAN
import os

In [3]:
df_train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
df_test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')

In [4]:
# Defining the age groups (5-12, 13-18, 19-22)
age_bins = [5, 13, 19, 23]
age_labels = ['5-12', '13-18', '19-22']

# Creating a new column for age groups
df_train['Age_Group'] = pd.cut(df_train['Basic_Demos-Age'], bins=age_bins, labels=age_labels, right=False)

# Creating a new column for age_band
df_train['Age_band'] = df_train['Age_Group'].map({'5-12': 0, '13-18': 1, '19-22': 2})
df_train['Age_band'].value_counts()

#단위변환한 변수가 있는 변수들은 제거
df_train.drop('Age_Group', axis = 1, inplace = True, errors = 'ignore')

In [5]:
drop_columns = ['BIA-Season',
 'Basic_Demos-Enroll_Season',
 'CGAS-Season',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_GSD',
 'FGC-FGC_GSD_Zone',
 'FGC-FGC_GSND',
 'FGC-FGC_GSND_Zone',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_SRL_Zone',
 'FGC-FGC_SRR_Zone',
 'FGC-FGC_TL_Zone',
 'FGC-Season',
 'Fitness_Endurance-Max_Stage',
 'Fitness_Endurance-Season',
 'Fitness_Endurance-Time_Mins',
 'Fitness_Endurance-Time_Sec',
 'PAQ_A-PAQ_A_Total',
 'PAQ_A-Season',
 'PAQ_C-PAQ_C_Total',
 'PAQ_C-Season',
 'PCIAT-PCIAT_01',
 'PCIAT-PCIAT_02',
 'PCIAT-PCIAT_03',
 'PCIAT-PCIAT_04',
 'PCIAT-PCIAT_05',
 'PCIAT-PCIAT_06',
 'PCIAT-PCIAT_07',
 'PCIAT-PCIAT_08',
 'PCIAT-PCIAT_09',
 'PCIAT-PCIAT_10',
 'PCIAT-PCIAT_11',
 'PCIAT-PCIAT_12',
 'PCIAT-PCIAT_13',
 'PCIAT-PCIAT_14',
 'PCIAT-PCIAT_15',
 'PCIAT-PCIAT_16',
 'PCIAT-PCIAT_17',
 'PCIAT-PCIAT_18',
 'PCIAT-PCIAT_19',
 'PCIAT-PCIAT_20',
 'PCIAT-PCIAT_Total',
 'PCIAT-Season',
 'Physical-Season',
 'PreInt_EduHx-Season',
 'SDS-SDS_Total_Raw',
 'SDS-Season',
 'sii']



# 피처 제거
df_train.drop(columns=drop_columns, inplace=True, errors='ignore')

# lbs -> kg 단위 변환
df_train['Physical-Weight(kg)'] = df_train['Physical-Weight'] * 0.453592

# Inch → cm 로 변환
df_train['Physical-Height(cm)'] = df_train['Physical-Height'] * 2.54

# 단위변환 이전 colum 제거
df_train = df_train.drop(columns=['Physical-Height', 'Physical-Weight'],  errors='ignore')

In [6]:
# Outlier
# 'Physical-Weight' 이상치 제거
age_q1 = []
age_q3 = []
age_iqr = []

for i in range(5, 19):
    # 각 나이에 대한 1사분위수(Q1)와 3사분위수(Q3) 계산
    i_q1 = df_train.loc[df_train['Basic_Demos-Age'] == i, 'Physical-Weight(kg)'].quantile(0.25)
    i_q3 = df_train.loc[df_train['Basic_Demos-Age'] == i, 'Physical-Weight(kg)'].quantile(0.75)
    
    # Q1, Q3, IQR 값을 각각의 리스트에 추가
    age_q1.append(i_q1)
    age_q3.append(i_q3)
    age_iqr.append(i_q3 - i_q1)

# 나이를 인덱스로 하는 DataFrame으로 변환하고 인덱스 이름을 'age'로 설정
age_iqr_df = pd.DataFrame({
    'Q1': age_q1,
    'Q3': age_q3,
    'IQR': age_iqr
}, index=range(5, 19))

age_iqr_df.index.name = 'Age'


# 나이가 19-22세인 사람들의 Q1, Q3, IQR 계산
age_19_22_q1 = df_train.loc[df_train['Basic_Demos-Age'].between(19, 22), 'Physical-Weight(kg)'].quantile(0.25)
age_19_22_q3 = df_train.loc[df_train['Basic_Demos-Age'].between(19, 22), 'Physical-Weight(kg)'].quantile(0.75)
age_19_22_iqr = age_19_22_q3 - age_19_22_q1

# 나이대 19-22세 행을 추가한 새로운 데이터프레임 생성
age_iqr_df.loc['19-22'] = [age_19_22_q1, age_19_22_q3, age_19_22_iqr]


for age in range(5, 19):
    q1 = age_iqr_df.loc[age, 'Q1']
    q3 = age_iqr_df.loc[age, 'Q3']
    iqr = age_iqr_df.loc[age, 'IQR']
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    df_train.loc[(df_train['Basic_Demos-Age'] == age) & 
                 ((df_train['Physical-Weight(kg)'] < lower_bound) | (df_train['Physical-Weight(kg)'] > upper_bound)), 
                 'Physical-Weight(kg)'] = None
    
q1_19_22 = age_iqr_df.loc['19-22', 'Q1']
q3_19_22 = age_iqr_df.loc['19-22', 'Q3']
iqr_19_22 = age_iqr_df.loc['19-22', 'IQR']
lower_bound_19_22 = q1_19_22 - 1.5 * iqr_19_22
upper_bound_19_22 = q3_19_22 + 1.5 * iqr_19_22

df_train.loc[(df_train['Basic_Demos-Age'].between(19, 22)) & 
             ((df_train['Physical-Weight(kg)'] < lower_bound_19_22) | (df_train['Physical-Weight(kg)'] > upper_bound_19_22)), 
             'Physical-Weight(kg)'] = None

# 'Physical-Height' 이상치 제거 
age_q1 = []
age_q3 = []
age_iqr = []

for i in range(5, 19):
    # 각 나이에 대한 1사분위수(Q1)와 3사분위수(Q3) 계산
    i_q1 = df_train.loc[df_train['Basic_Demos-Age'] == i, 'Physical-Height(cm)'].quantile(0.25)
    i_q3 = df_train.loc[df_train['Basic_Demos-Age'] == i, 'Physical-Height(cm)'].quantile(0.75)
    
    # Q1, Q3, IQR 값을 각각의 리스트에 추가
    age_q1.append(i_q1)
    age_q3.append(i_q3)
    age_iqr.append(i_q3 - i_q1)

# 나이를 인덱스로 하는 DataFrame으로 변환하고 인덱스 이름을 'age'로 설정
age_iqr_df = pd.DataFrame({
    'Q1': age_q1,
    'Q3': age_q3,
    'IQR': age_iqr
}, index=range(5, 19))

age_iqr_df.index.name = 'Age'


# 나이가 19-22세인 사람들의 Q1, Q3, IQR 계산
age_19_22_q1 = df_train.loc[df_train['Basic_Demos-Age'].between(19, 22), 'Physical-Height(cm)'].quantile(0.25)
age_19_22_q3 = df_train.loc[df_train['Basic_Demos-Age'].between(19, 22), 'Physical-Height(cm)'].quantile(0.75)
age_19_22_iqr = age_19_22_q3 - age_19_22_q1

# 나이대 19-22세 행을 추가한 새로운 데이터프레임 생성
age_iqr_df.loc['19-22'] = [age_19_22_q1, age_19_22_q3, age_19_22_iqr]


for age in range(5, 19):
    q1 = age_iqr_df.loc[age, 'Q1']
    q3 = age_iqr_df.loc[age, 'Q3']
    iqr = age_iqr_df.loc[age, 'IQR']
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Applying the outlier bounds to replace values outside of this range with NaN
    df_train.loc[(df_train['Basic_Demos-Age'] == age) & 
                 ((df_train['Physical-Height(cm)'] < lower_bound) | (df_train['Physical-Height(cm)'] > upper_bound)), 
                 'Physical-Height(cm)'] = None
    
q1_19_22 = age_iqr_df.loc['19-22', 'Q1']
q3_19_22 = age_iqr_df.loc['19-22', 'Q3']
iqr_19_22 = age_iqr_df.loc['19-22', 'IQR']
lower_bound_19_22 = q1_19_22 - 1.5 * iqr_19_22
upper_bound_19_22 = q3_19_22 + 1.5 * iqr_19_22

df_train.loc[(df_train['Basic_Demos-Age'].between(19, 22)) & 
             ((df_train['Physical-Height(cm)'] < lower_bound_19_22) | (df_train['Physical-Height(cm)'] > upper_bound_19_22)), 
             'Physical-Height(cm)'] = None


# 'Physical-BMI' 이상치 제거 
df_train.loc[(df_train['Physical-BMI'].notnull()) & 
             (df_train['Physical-Weight(kg)'].isnull() | df_train['Physical-Height(cm)'].isnull()), 
             'Physical-BMI'] = None

df_train['Physical-BMI(calc)'] = df_train['Physical-Weight(kg)'] / (df_train['Physical-Height(cm)'] / 100) ** 2
df_train['BMI_diff'] = abs(df_train['Physical-BMI(calc)'] - df_train['Physical-BMI'])

#단위변환한 변수가 있는 변수들은 제거
df_train.drop(['Physical-BMI', 'BMI_diff'], axis = 1, inplace = True, errors = 'ignore')


# Physical-Waist_Circumference 이상치 제거
age_q1 = []
age_q3 = []
age_iqr = []

for i in range(5, 19):
    # 각 나이에 대한 1사분위수(Q1)와 3사분위수(Q3) 계산
    i_q1 = df_train.loc[df_train['Basic_Demos-Age'] == i, 'Physical-Waist_Circumference'].quantile(0.25)
    i_q3 = df_train.loc[df_train['Basic_Demos-Age'] == i, 'Physical-Waist_Circumference'].quantile(0.75)
    
    # Q1, Q3, IQR 값을 각각의 리스트에 추가
    age_q1.append(i_q1)
    age_q3.append(i_q3)
    age_iqr.append(i_q3 - i_q1)

# 나이를 인덱스로 하는 DataFrame으로 변환하고 인덱스 이름을 'age'로 설정
age_iqr_df = pd.DataFrame({
    'Q1': age_q1,
    'Q3': age_q3,
    'IQR': age_iqr
}, index=range(5, 19))

age_iqr_df.index.name = 'Age'



# 나이가 19-22세인 사람들의 Q1, Q3, IQR 계산
age_19_22_q1 = df_train.loc[df_train['Basic_Demos-Age'].between(19, 22), 'Physical-Waist_Circumference'].quantile(0.25)
age_19_22_q3 = df_train.loc[df_train['Basic_Demos-Age'].between(19, 22), 'Physical-Waist_Circumference'].quantile(0.75)
age_19_22_iqr = age_19_22_q3 - age_19_22_q1

# 나이대 19-22세 행을 추가한 새로운 데이터프레임 생성
age_iqr_df.loc['19-22'] = [age_19_22_q1, age_19_22_q3, age_19_22_iqr]


for age in range(5, 19):
    q1 = age_iqr_df.loc[age, 'Q1']
    q3 = age_iqr_df.loc[age, 'Q3']
    iqr = age_iqr_df.loc[age, 'IQR']
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Applying the outlier bounds to replace values outside of this range with NaN
    df_train.loc[(df_train['Basic_Demos-Age'] == age) & 
                 ((df_train['Physical-Waist_Circumference'] < lower_bound) | (df_train['Physical-Waist_Circumference'] > upper_bound)), 
                 'Physical-Waist_Circumference'] = None
    
q1_19_22 = age_iqr_df.loc['19-22', 'Q1']
q3_19_22 = age_iqr_df.loc['19-22', 'Q3']
iqr_19_22 = age_iqr_df.loc['19-22', 'IQR']
lower_bound_19_22 = q1_19_22 - 1.5 * iqr_19_22
upper_bound_19_22 = q3_19_22 + 1.5 * iqr_19_22

df_train.loc[(df_train['Basic_Demos-Age'].between(19, 22)) & 
             ((df_train['Physical-Waist_Circumference'] < lower_bound_19_22) | (df_train['Physical-Waist_Circumference'] > upper_bound_19_22)), 
             'Physical-Waist_Circumference'] = None

# Physical-HeartRate 이상치 제거
age_band_q1 = []
age_band_q3 = []
age_band_iqr = []

for i in range(0, 3):
    i_q1 = df_train.loc[df_train['Age_band'] == i, 'Physical-HeartRate'].quantile(0.25)
    i_q3 = df_train.loc[df_train['Age_band'] == i, 'Physical-HeartRate'].quantile(0.75)
    
    # Q1, Q3, IQR 값을 각각의 리스트에 추가
    age_band_q1.append(i_q1)
    age_band_q3.append(i_q3)
    age_band_iqr.append(i_q3 - i_q1)

# 나이를 인덱스로 하는 DataFrame으로 변환하고 인덱스 이름을 'age'로 설정
age_band_iqr = pd.DataFrame({
    'Q1': age_band_q1,
    'Q3': age_band_q3,
    'IQR': age_band_iqr
}, index=range(0, 3))

age_band_iqr.index.name = 'Age_band'


for age_band in range(0, 3):
    q1 = age_band_iqr.loc[age_band, 'Q1']
    q3 = age_band_iqr.loc[age_band, 'Q3']
    iqr = age_band_iqr.loc[age_band, 'IQR']
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Applying the outlier bounds to replace values outside of this range with NaN
    df_train.loc[(df_train['Age_band'] == age_band) & 
                 ((df_train['Physical-HeartRate'] < lower_bound) | (df_train['Physical-HeartRate'] > upper_bound)), 
                 'Physical-HeartRate'] = None

# Physical-Systolic_BP, Physical-Diastolic_BP 이상치 제거
rows_to_swap = df_train['Physical-Systolic_BP'] < df_train['Physical-Diastolic_BP']
df_train.loc[rows_to_swap, ['Physical-Systolic_BP', 'Physical-Diastolic_BP']]


# 수축기 혈압이 이완기 혈압보다 낮은 Case는 3가지. 이는 말이 안되므로 3가지 case는 수치 변경함

# In[53]:


rows_to_swap = df_train['Physical-Systolic_BP'] < df_train['Physical-Diastolic_BP']
df_train.loc[rows_to_swap, ['Physical-Systolic_BP', 'Physical-Diastolic_BP']] = df_train.loc[rows_to_swap, ['Physical-Diastolic_BP', 'Physical-Systolic_BP']].values


# ## Age_band 별로 IQR 구하고 이상치 제거 - 'Physical-Systolic_BP'
df_train.groupby('Age_band')['Physical-Systolic_BP'].mean()


age_band_q1 = []
age_band_q3 = []
age_band_iqr = []

for i in range(0, 3):
    i_q1 = df_train.loc[df_train['Age_band'] == i, 'Physical-Systolic_BP'].quantile(0.25)
    i_q3 = df_train.loc[df_train['Age_band'] == i, 'Physical-Systolic_BP'].quantile(0.75)
    
    # Q1, Q3, IQR 값을 각각의 리스트에 추가
    age_band_q1.append(i_q1)
    age_band_q3.append(i_q3)
    age_band_iqr.append(i_q3 - i_q1)

# 나이를 인덱스로 하는 DataFrame으로 변환하고 인덱스 이름을 'age'로 설정
age_band_iqr = pd.DataFrame({
    'Q1': age_band_q1,
    'Q3': age_band_q3,
    'IQR': age_band_iqr
}, index=range(0, 3))

age_band_iqr.index.name = 'Age_band'


for age_band in range(0, 3):
    q1 = age_band_iqr.loc[age_band, 'Q1']
    q3 = age_band_iqr.loc[age_band, 'Q3']
    iqr = age_band_iqr.loc[age_band, 'IQR']
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Applying the outlier bounds to replace values outside of this range with NaN
    df_train.loc[(df_train['Age_band'] == age_band) & 
                 ((df_train['Physical-Systolic_BP'] < lower_bound) | (df_train['Physical-Systolic_BP'] > upper_bound)), 
                 'Physical-Systolic_BP'] = None




# ## Age_band 별로 IQR 구하고 이상치 제거 - 'Physical-Diastolic_BP'

age_band_q1 = []
age_band_q3 = []
age_band_iqr = []

for i in range(0, 3):
    i_q1 = df_train.loc[df_train['Age_band'] == i, 'Physical-Diastolic_BP'].quantile(0.25)
    i_q3 = df_train.loc[df_train['Age_band'] == i, 'Physical-Diastolic_BP'].quantile(0.75)
    
    # Q1, Q3, IQR 값을 각각의 리스트에 추가
    age_band_q1.append(i_q1)
    age_band_q3.append(i_q3)
    age_band_iqr.append(i_q3 - i_q1)

# 나이를 인덱스로 하는 DataFrame으로 변환하고 인덱스 이름을 'age'로 설정
age_band_iqr = pd.DataFrame({
    'Q1': age_band_q1,
    'Q3': age_band_q3,
    'IQR': age_band_iqr
}, index=range(0, 3))

age_band_iqr.index.name = 'Age_band'

for age_band in range(0, 3):
    q1 = age_band_iqr.loc[age_band, 'Q1']
    q3 = age_band_iqr.loc[age_band, 'Q3']
    iqr = age_band_iqr.loc[age_band, 'IQR']
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Applying the outlier bounds to replace values outside of this range with NaN
    df_train.loc[(df_train['Age_band'] == age_band) & 
                 ((df_train['Physical-Diastolic_BP'] < lower_bound) | (df_train['Physical-Diastolic_BP'] > upper_bound)), 
                 'Physical-Diastolic_BP'] = None


# FGC Data
fgc_columns = [col for col in df_train.columns if 'FGC-FGC' in col]
df_train[fgc_columns].describe()



# Initialize lists to store Q1, Q3, and IQR values for each age_band for all FGC columns
fgc_stats = {col: {'Q1': [], 'Q3': [], 'IQR': []} for col in fgc_columns}

# Calculate Q1, Q3, and IQR for each age_band for each FGC column
for col in fgc_columns:
    for i in range(0, 3):  # Assuming Age_band ranges from 0 to 2
        q1 = df_train.loc[df_train['Age_band'] == i, col].quantile(0.25)
        q3 = df_train.loc[df_train['Age_band'] == i, col].quantile(0.75)
        iqr = q3 - q1
        
        # Append the calculated values to the corresponding lists
        fgc_stats[col]['Q1'].append(q1)
        fgc_stats[col]['Q3'].append(q3)
        fgc_stats[col]['IQR'].append(iqr)

# Apply outlier bounds and replace outliers with NaN
for col in fgc_columns:
    for age_band in range(0, 3):
        q1 = fgc_stats[col]['Q1'][age_band]
        q3 = fgc_stats[col]['Q3'][age_band]
        iqr = fgc_stats[col]['IQR'][age_band]
        lower_bound = q1 - 3 * iqr
        upper_bound = q3 + 3 * iqr

        # Replace values outside the bounds with NaN
        df_train.loc[(df_train['Age_band'] == age_band) & 
                     ((df_train[col] < lower_bound) | (df_train[col] > upper_bound)), col] = None


# BIA Data
bia_columns = [col for col in df_train.columns if col.startswith('BIA-BIA')]

# 나이별로 각 'BIA-BIA' 열의 Q1, Q3, IQR을 계산하고 이상치를 결측치로 변환
for column in bia_columns:
    # 나이별로 Q1, Q3, IQR을 저장할 데이터프레임 생성
    age_q1 = []
    age_q3 = []
    age_iqr = []
    
    for age in range(5, 19):
        i_q1 = df_train.loc[df_train['Basic_Demos-Age'] == age, column].quantile(0.25)
        i_q3 = df_train.loc[df_train['Basic_Demos-Age'] == age, column].quantile(0.75)
        age_q1.append(i_q1)
        age_q3.append(i_q3)
        age_iqr.append(i_q3 - i_q1)
    
    # 데이터프레임 생성하여 나이별 Q1, Q3, IQR 값 저장
    age_iqr_df = pd.DataFrame({
        'Q1': age_q1,
        'Q3': age_q3,
        'IQR': age_iqr
    }, index=range(5, 19))
    
    # 19-22세 나이 그룹에 대한 Q1, Q3, IQR 계산 및 데이터프레임에 추가
    age_19_22_q1 = df_train.loc[df_train['Basic_Demos-Age'].between(19, 22), column].quantile(0.25)
    age_19_22_q3 = df_train.loc[df_train['Basic_Demos-Age'].between(19, 22), column].quantile(0.75)
    age_19_22_iqr = age_19_22_q3 - age_19_22_q1
    age_iqr_df.loc['19-22'] = [age_19_22_q1, age_19_22_q3, age_19_22_iqr]
    
    # 각 나이대에 대해 IQR 기준으로 이상치를 결측치로 대체
    for age in range(5, 19):
        q1 = age_iqr_df.loc[age, 'Q1']
        q3 = age_iqr_df.loc[age, 'Q3']
        iqr = age_iqr_df.loc[age, 'IQR']
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        
        # 이상치 범위를 벗어난 값들을 None으로 대체
        df_train.loc[(df_train['Basic_Demos-Age'] == age) & 
                     ((df_train[column] < lower_bound) | (df_train[column] > upper_bound)), column] = None
    
    # 19-22세 그룹 이상치 처리
    lower_bound_19_22 = age_19_22_q1 - 1.5 * age_19_22_iqr
    upper_bound_19_22 = age_19_22_q3 + 1.5 * age_19_22_iqr
    
    df_train.loc[(df_train['Basic_Demos-Age'].between(19, 22)) & 
                 ((df_train[column] < lower_bound_19_22) | (df_train[column] > upper_bound_19_22)), column] = None



In [7]:
# KNN_sequential

numeric_cols = df_train.select_dtypes(include=['int64', 'float64']).columns

# Step 3: Separate numeric and non-numeric columns
non_numeric_cols = df_train.select_dtypes(exclude=['int64', 'float64']).columns
numeric_data = df_train[numeric_cols]
non_numeric_data = df_train[non_numeric_cols]

# Step 4: Calculate missing values ratio for numeric columns
missing_ratios = numeric_data.isnull().mean().sort_values()

# Ensure 'PreInt_EduHx-computerinternet_hoursday' is imputed last
if 'PreInt_EduHx-computerinternet_hoursday' in missing_ratios.index:
    missing_ratios = missing_ratios.drop('PreInt_EduHx-computerinternet_hoursday')
    missing_ratios['PreInt_EduHx-computerinternet_hoursday'] = 1.0  # Set it as the last column to impute

# Step 5: Sequentially impute missing values using KNN Imputer
imputer = KNNImputer(n_neighbors=4)
numeric_data_imputed = numeric_data.copy()

for col in tqdm(missing_ratios.index, desc="Columns Progress"):
    if numeric_data[col].isnull().sum() > 0:
        # Impute the target column using KNN Imputer
        numeric_data_imputed[col] = imputer.fit_transform(numeric_data_imputed)[
            :, numeric_data.columns.get_loc(col)
        ]

# Step 6: Combine imputed numeric data with non-numeric data
df_imputed = pd.concat([numeric_data_imputed, non_numeric_data], axis=1)

# Step 7: Convert integer-like float columns back to int where necessary
for col in numeric_data_imputed.select_dtypes(include=['float64']).columns:
    if col in df_train.columns and df_train[col].dtype == 'int64':
        df_imputed[col] = df_imputed[col].round().astype(int)

# Step 8: Round float columns to 2 decimal places
for col in numeric_data_imputed.select_dtypes(include=['float64']).columns:
    if col in df_train.columns and df_train[col].dtype == 'float64':
        df_imputed[col] = df_imputed[col].round(2)

# Step 9: Verify that all missing values have been filled
print(df_imputed.isnull().sum())

df_train = df_imputed
train_imputed = df_imputed

Columns Progress: 100%|██████████| 33/33 [01:10<00:00,  2.12s/it]

Basic_Demos-Age                           0
Basic_Demos-Sex                           0
CGAS-CGAS_Score                           0
Physical-Waist_Circumference              0
Physical-Diastolic_BP                     0
Physical-HeartRate                        0
Physical-Systolic_BP                      0
FGC-FGC_CU                                0
FGC-FGC_PU                                0
FGC-FGC_SRL                               0
FGC-FGC_SRR                               0
FGC-FGC_TL                                0
BIA-BIA_Activity_Level_num                0
BIA-BIA_BMC                               0
BIA-BIA_BMI                               0
BIA-BIA_BMR                               0
BIA-BIA_DEE                               0
BIA-BIA_ECW                               0
BIA-BIA_FFM                               0
BIA-BIA_FFMI                              0
BIA-BIA_FMI                               0
BIA-BIA_Fat                               0
BIA-BIA_Frame_num               

In [8]:
# PCA

#단위변환한 변수가 있는 변수들은 제거
train_imputed.drop(['Age_Group','Physical-BMI', 'Physical-Height','Physical-Weight', 'BIA-BIA_BMI'], axis = 1, inplace = True, errors = 'ignore')

# 'id' 열을 제외한 상태로 처리
id_column = train_imputed['id']  # 'id' 열 따로 저장

# Label encode categorical columns (excluding 'id')
categorical_cols = train_imputed.select_dtypes(include=['object']).columns.difference(['id'])
label_encoder = LabelEncoder()


# 선택한 특성 추출
train_imputed['FGC-FGC_SR'] = (train_imputed['FGC-FGC_SRL'] + train_imputed['FGC-FGC_SRR'])/2
FGC_SR = train_imputed[['FGC-FGC_SRL', 'FGC-FGC_SRR']]

# 데이터 표준화
scaler = StandardScaler()
features_scaled = scaler.fit_transform(FGC_SR)

# PCA 모델 설정, 주성분 개수를 1로 설정
pca = PCA(n_components=1)

# PCA 적용
FGC_SR = pca.fit_transform(FGC_SR)

# 설명된 분산 비율 확인
explained_variance = pca.explained_variance_ratio_

#Sit_Reach 주성분을 데이터 프레임으로 만들기
FGC_SR_df = pd.DataFrame(data = FGC_SR, columns = ['FGC_SR'])




# 선택한 특성 추출
BIA_features = train_imputed[['BIA-BIA_BMC','BIA-BIA_FMI','BIA-BIA_BMR','BIA-BIA_DEE', 'BIA-BIA_ECW','BIA-BIA_FFM','BIA-BIA_FFMI','BIA-BIA_Fat','BIA-BIA_ICW','BIA-BIA_LDM','BIA-BIA_LST','BIA-BIA_SMM','BIA-BIA_TBW']]

# 데이터 표준화
scaler = StandardScaler()
features_scaled = scaler.fit_transform(BIA_features)

# PCA 모델 설정
pca = PCA(n_components=1)

# PCA 적용
BIA = pca.fit_transform(BIA_features)

# 설명된 분산 비율 확인
explained_variance = pca.explained_variance_ratio_

#BIA 주성분을 데이터 프레임으로 만들기
BIA_df = pd.DataFrame(data = BIA, columns = ['BIA'])

# Ensure 'id' column is intact in the final concatenated DataFrame
train_concat = pd.concat([train_imputed, FGC_SR_df, BIA_df], axis=1)

#PCA를 통해 축소한 feature들을 제거
train_concat.drop(['BMI_diff','FGC-FGC_SR', 'FGC-FGC_SRL','FGC-FGC_SRR','BIA-BIA_BMC', 'BIA-BIA_BMR','BIA-BIA_DEE', 'BIA-BIA_ECW','BIA-BIA_FFM','BIA-BIA_FFMI','BIA-BIA_Fat','BIA-BIA_ICW','BIA-BIA_LDM','BIA-BIA_LST','BIA-BIA_SMM','BIA-BIA_TBW','BIA-BIA_FMI'], axis =1, inplace = True, errors = 'ignore')

#PCIAT_Total Column을 추가하는 코드
df_pciat = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
df_pciat_total = df_pciat.loc[:,'PCIAT-PCIAT_Total']
train_concat1 = pd.concat([train_concat, df_pciat_total], axis = 1)

#처리한 df를 df_train이름으로 변경
df_train = train_concat1

In [9]:
# semi-supervied learning for PCIAT_Total Missing with Multiview

df_train['Age_band'] = df_train['Age_band'].astype(int)
target_col = 'PCIAT-PCIAT_Total'

# 라벨이 있는 데이터와 없는 데이터 분리
df_labeled = df_train[df_train[target_col].notnull()]
df_unlabeled = df_train[df_train[target_col].isnull()]


# 두 개의 뷰로 데이터 분리 (id 포함)
view1_cols = [
    "id", "Basic_Demos-Age", "Basic_Demos-Sex", "Physical-Waist_Circumference", 
    "Physical-Diastolic_BP", "Physical-HeartRate", "Physical-Systolic_BP", 
    "BIA-BIA_Activity_Level_num", "BIA-BIA_Frame_num", "Physical-BMI(calc)", 
    "Age_band", "BIA"
]

view2_cols = [
    "id", "CGAS-CGAS_Score", "FGC-FGC_CU", "FGC-FGC_PU", "FGC-FGC_TL", 
    "SDS-SDS_Total_T", "PreInt_EduHx-computerinternet_hoursday", 
    "FGC_SR"
]

# Split labeled and unlabeled data by the views
X1_labeled = df_labeled[view1_cols]
X2_labeled = df_labeled[view2_cols]

X1_unlabeled = df_unlabeled[view1_cols]
X2_unlabeled = df_unlabeled[view2_cols]

# Step 2: 모델 초기화
# 'id' 열을 제외한 데이터로 학습
X1_train = X1_labeled.drop(columns=["id"])
X2_train = X2_labeled.drop(columns=["id"])
y_train = df_labeled[target_col]

# Initialize XGBoost models
model1 = XGBRegressor(random_state=42)
model2 = XGBRegressor(random_state=42)

# Train the models
model1.fit(X1_train, y_train)
model2.fit(X2_train, y_train)

# Drop 'id' column for prediction
X1_unlabeled_features = X1_unlabeled.drop(columns=["id"])
X2_unlabeled_features = X2_unlabeled.drop(columns=["id"])

# Predict using both models
preds1_unlabeled = model1.predict(X1_unlabeled_features)
preds2_unlabeled = model2.predict(X2_unlabeled_features)


# Add 'id' back to the predictions
preds1_with_id = pd.DataFrame({
    "id": X1_unlabeled["id"].values,
    "preds1": preds1_unlabeled
})

preds2_with_id = pd.DataFrame({
    "id": X2_unlabeled["id"].values,
    "preds2": preds2_unlabeled
})

preds_merged = pd.merge(preds1_with_id, preds2_with_id, on="id", how="inner")
preds_merged['Diff'] = abs(preds_merged['preds1'] - preds_merged['preds2'])
preds_merged['Average'] = (0.5 * (preds_merged['preds1'] + preds_merged['preds2'])).astype(int)
preds_merged.head()


# Filter rows where Diff <= 10
filtered_preds = preds_merged[preds_merged['Diff'] <= 10]

# Update PCIAT-PCIAT_Total in df_train using id as the key
df_train.loc[
    df_train['id'].isin(filtered_preds['id']),
    'PCIAT-PCIAT_Total'
] = df_train['id'].map(
    filtered_preds.set_index('id')['Average']
)
df_train = df_train.dropna(subset=['PCIAT-PCIAT_Total'])

In [10]:
## CTGAN 으로 데이터 증강
# 0-30=None; 31-49=Mild; 50-79=Moderate; 80-100=Severe
def determine_sii(value):
    # 음수 xgb에서 좀 발생함
    if value < 0:
        value = 0    
    if 0 <= value < 30:
        return 0  
    elif 30 <= value < 50:
        return 1  
    elif 50 <= value < 80:
        return 2  
    elif 80 <= value <= 100:
        return 3
    elif value > 100:
        return 3
    else:
        print(f"문제가 생겼다 - {value}")
        return None
        
#df_train['sii'] = df_train['PCIAT-PCIAT_Total'].apply(determine_sii)

In [11]:
df_train = df_train.drop(columns = 'id', errors = 'ignore')
df_train

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,FGC-FGC_CU,FGC-FGC_PU,FGC-FGC_TL,...,BIA-BIA_Frame_num,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-Weight(kg),Physical-Height(cm),Physical-BMI(calc),Age_band,FGC_SR,BIA,PCIAT-PCIAT_Total
0,5,0,51.00,21.50,60.0,89.50,105.50,0.00,0.0,6.0,...,1.00,52.75,3.00,23.04,116.84,16.88,0,-3.200509,-501.736246,55.0
1,9,0,61.50,22.00,75.0,70.00,122.00,3.00,5.0,3.0,...,1.00,64.00,0.00,20.87,121.92,14.04,0,3.160941,-494.367571,0.0
2,10,1,71.00,28.00,65.0,94.00,117.00,20.00,7.0,5.0,...,1.75,54.00,2.00,34.29,143.51,16.65,0,1.746736,-157.875173,28.0
3,9,0,71.00,26.50,60.0,97.00,117.00,18.00,5.0,7.0,...,2.00,45.00,0.00,37.01,142.24,18.29,0,-2.495879,-25.323968,44.0
4,18,1,59.25,35.50,67.0,72.75,121.75,13.75,6.0,11.0,...,2.00,59.00,1.75,64.95,162.45,24.27,1,1.744264,86.945164,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,8,0,63.50,25.00,60.0,65.00,112.00,0.00,0.0,12.0,...,1.00,58.00,2.00,30.48,133.35,17.14,0,0.327588,-214.736264,22.0
3954,7,1,60.00,23.00,65.0,75.00,105.00,0.00,0.0,4.5,...,1.00,67.00,0.00,21.14,123.19,13.93,0,-0.019784,-702.683669,33.0
3955,13,0,60.00,26.00,71.0,70.00,104.00,16.00,10.0,12.0,...,2.00,50.00,1.00,37.38,151.13,16.36,1,-0.377043,122.992874,32.0
3957,11,0,68.00,29.25,79.0,99.00,116.00,15.00,0.0,14.0,...,2.00,77.00,0.00,49.80,152.40,21.44,0,1.746736,101.410844,31.0


In [12]:
import optuna
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
import numpy as np

# 입력 데이터 설정
X = df_train.drop(columns=['PCIAT-PCIAT_Total'])
y = df_train['PCIAT-PCIAT_Total']

# determine_sii 함수로 y 범주형 변환
y_stratified = y.apply(determine_sii)

# Stratified K-Fold 설정
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

# Optuna를 사용한 하이퍼파라미터 튜닝
def objective(trial):
    # 하이퍼파라미터 공간 설정
    params = {
        'iterations': trial.suggest_int('iterations', 500, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'depth': trial.suggest_int('depth', 4, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 15),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'loss_function': 'RMSE',
        'random_seed': 42,
        'verbose': 0  # 로그 출력 제한
    }

    # Stratified K-Fold를 사용한 교차 검증
    rmse_scores = []
    for train_index, valid_index in skf.split(X, y_stratified):
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        # CatBoost 모델 학습
        model = CatBoostRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=(X_valid, y_valid),
            early_stopping_rounds=50,
            verbose=0
        )

        # 검증 데이터 예측 및 RMSE 계산
        y_pred = model.predict(X_valid)
        rmse = mean_squared_error(y_valid, y_pred, squared=False)
        rmse_scores.append(rmse)

    # 평균 RMSE 반환
    return np.mean(rmse_scores)

# Optuna 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # 시도 횟수 증가

# 최적 파라미터 출력
print("Best hyperparameters:", study.best_params)

[I 2024-12-17 01:57:24,067] A new study created in memory with name: no-name-67e67f14-d3b1-43dc-9520-34457c3873c5
[I 2024-12-17 01:57:25,850] Trial 0 finished with value: 15.620170684339335 and parameters: {'iterations': 1900, 'learning_rate': 0.17539633637131458, 'depth': 8, 'l2_leaf_reg': 7.35137590785877, 'subsample': 0.8880988155016845}. Best is trial 0 with value: 15.620170684339335.
[I 2024-12-17 01:57:32,457] Trial 1 finished with value: 15.454454302029712 and parameters: {'iterations': 800, 'learning_rate': 0.034756428419178184, 'depth': 8, 'l2_leaf_reg': 10.146338628754723, 'subsample': 0.900950290852672}. Best is trial 1 with value: 15.454454302029712.
[I 2024-12-17 01:57:33,596] Trial 2 finished with value: 15.556318028258325 and parameters: {'iterations': 1600, 'learning_rate': 0.05565831887732851, 'depth': 4, 'l2_leaf_reg': 2.915793829428341, 'subsample': 0.7865437364292718}. Best is trial 1 with value: 15.454454302029712.
[I 2024-12-17 01:57:40,187] Trial 3 finished with 

Best hyperparameters: {'iterations': 800, 'learning_rate': 0.034756428419178184, 'depth': 8, 'l2_leaf_reg': 10.146338628754723, 'subsample': 0.900950290852672}


In [13]:
# 최적 파라미터 설정
best_params = study.best_params

# 최종 모델 학습 (모든 데이터를 사용)
final_model = CatBoostRegressor(**best_params, loss_function='RMSE', verbose=100)
final_model.fit(X, y)

0:	learn: 18.8181923	total: 8.3ms	remaining: 6.63s
100:	learn: 14.4156892	total: 499ms	remaining: 3.45s
200:	learn: 13.4029469	total: 940ms	remaining: 2.8s
300:	learn: 12.9083008	total: 1.37s	remaining: 2.27s
400:	learn: 12.5854640	total: 1.78s	remaining: 1.78s
500:	learn: 12.1045302	total: 2.22s	remaining: 1.33s
600:	learn: 11.5376987	total: 2.66s	remaining: 881ms
700:	learn: 10.9735939	total: 3.08s	remaining: 435ms
799:	learn: 10.4256735	total: 3.52s	remaining: 0us


In [14]:
df_test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')

In [15]:
drop_columns = ['BIA-Season',
 'Basic_Demos-Enroll_Season',
 'CGAS-Season',
 'FGC-FGC_CU_Zone',
 'FGC-FGC_GSD',
 'FGC-FGC_GSD_Zone',
 'FGC-FGC_GSND',
 'FGC-FGC_GSND_Zone',
 'FGC-FGC_PU_Zone',
 'FGC-FGC_SRL_Zone',
 'FGC-FGC_SRR_Zone',
 'FGC-FGC_TL_Zone',
 'FGC-Season',
 'Fitness_Endurance-Max_Stage',
 'Fitness_Endurance-Season',
 'Fitness_Endurance-Time_Mins',
 'Fitness_Endurance-Time_Sec',
 'PAQ_A-PAQ_A_Total',
 'PAQ_A-Season',
 'PAQ_C-PAQ_C_Total',
 'PAQ_C-Season',
 'PCIAT-PCIAT_01',
 'PCIAT-PCIAT_02',
 'PCIAT-PCIAT_03',
 'PCIAT-PCIAT_04',
 'PCIAT-PCIAT_05',
 'PCIAT-PCIAT_06',
 'PCIAT-PCIAT_07',
 'PCIAT-PCIAT_08',
 'PCIAT-PCIAT_09',
 'PCIAT-PCIAT_10',
 'PCIAT-PCIAT_11',
 'PCIAT-PCIAT_12',
 'PCIAT-PCIAT_13',
 'PCIAT-PCIAT_14',
 'PCIAT-PCIAT_15',
 'PCIAT-PCIAT_16',
 'PCIAT-PCIAT_17',
 'PCIAT-PCIAT_18',
 'PCIAT-PCIAT_19',
 'PCIAT-PCIAT_20',
 'PCIAT-PCIAT_Total',
 'PCIAT-Season',
 'Physical-Season',
 'PreInt_EduHx-Season',
 'SDS-SDS_Total_Raw',
 'SDS-Season',
 'sii']



# 피처 제거
df_test.drop(columns=drop_columns, inplace=True, errors='ignore')

In [16]:
# Defining the age groups (5-12, 13-18, 19-22)
age_bins = [5, 13, 19, 23]
age_labels = ['5-12', '13-18', '19-22']

# Creating a new column for age groups
df_test['Age_Group'] = pd.cut(df_test['Basic_Demos-Age'], bins=age_bins, labels=age_labels, right=False)

# Creating a new column for age_band
df_test['Age_band'] = df_test['Age_Group'].map({'5-12': 0, '13-18': 1, '19-22': 2})
df_test['Age_band'].value_counts()

#단위변환한 변수가 있는 변수들은 제거
df_test.drop('Age_Group', axis = 1, inplace = True, errors = 'ignore')

In [17]:
# test_Outlier

# lbs -> kg 단위 변환
df_test['Physical-Weight(kg)'] = df_test['Physical-Weight'] * 0.453592

# Inch → cm 로 변환
df_test['Physical-Height(cm)'] = df_test['Physical-Height'] * 2.54

# 단위변환 이전 colum 제거
df_test = df_test.drop(columns=['Physical-Height', 'Physical-Weight'],  errors='ignore')

# 'Physical-Weight' 이상치 제거
age_q1 = []
age_q3 = []
age_iqr = []

for i in range(5, 19):
    # 각 나이에 대한 1사분위수(Q1)와 3사분위수(Q3) 계산
    i_q1 = df_test.loc[df_test['Basic_Demos-Age'] == i, 'Physical-Weight(kg)'].quantile(0.25)
    i_q3 = df_test.loc[df_test['Basic_Demos-Age'] == i, 'Physical-Weight(kg)'].quantile(0.75)
    
    # Q1, Q3, IQR 값을 각각의 리스트에 추가
    age_q1.append(i_q1)
    age_q3.append(i_q3)
    age_iqr.append(i_q3 - i_q1)

# 나이를 인덱스로 하는 DataFrame으로 변환하고 인덱스 이름을 'age'로 설정
age_iqr_df = pd.DataFrame({
    'Q1': age_q1,
    'Q3': age_q3,
    'IQR': age_iqr
}, index=range(5, 19))

age_iqr_df.index.name = 'Age'


# 나이가 19-22세인 사람들의 Q1, Q3, IQR 계산
age_19_22_q1 = df_test.loc[df_test['Basic_Demos-Age'].between(19, 22), 'Physical-Weight(kg)'].quantile(0.25)
age_19_22_q3 = df_test.loc[df_test['Basic_Demos-Age'].between(19, 22), 'Physical-Weight(kg)'].quantile(0.75)
age_19_22_iqr = age_19_22_q3 - age_19_22_q1

# 나이대 19-22세 행을 추가한 새로운 데이터프레임 생성
age_iqr_df.loc['19-22'] = [age_19_22_q1, age_19_22_q3, age_19_22_iqr]


for age in range(5, 19):
    q1 = age_iqr_df.loc[age, 'Q1']
    q3 = age_iqr_df.loc[age, 'Q3']
    iqr = age_iqr_df.loc[age, 'IQR']
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    df_test.loc[(df_test['Basic_Demos-Age'] == age) & 
                 ((df_test['Physical-Weight(kg)'] < lower_bound) | (df_test['Physical-Weight(kg)'] > upper_bound)), 
                 'Physical-Weight(kg)'] = None
    
q1_19_22 = age_iqr_df.loc['19-22', 'Q1']
q3_19_22 = age_iqr_df.loc['19-22', 'Q3']
iqr_19_22 = age_iqr_df.loc['19-22', 'IQR']
lower_bound_19_22 = q1_19_22 - 1.5 * iqr_19_22
upper_bound_19_22 = q3_19_22 + 1.5 * iqr_19_22

df_test.loc[(df_test['Basic_Demos-Age'].between(19, 22)) & 
             ((df_test['Physical-Weight(kg)'] < lower_bound_19_22) | (df_test['Physical-Weight(kg)'] > upper_bound_19_22)), 
             'Physical-Weight(kg)'] = None

# 'Physical-Height' 이상치 제거 
age_q1 = []
age_q3 = []
age_iqr = []

for i in range(5, 19):
    # 각 나이에 대한 1사분위수(Q1)와 3사분위수(Q3) 계산
    i_q1 = df_test.loc[df_test['Basic_Demos-Age'] == i, 'Physical-Height(cm)'].quantile(0.25)
    i_q3 = df_test.loc[df_test['Basic_Demos-Age'] == i, 'Physical-Height(cm)'].quantile(0.75)
    
    # Q1, Q3, IQR 값을 각각의 리스트에 추가
    age_q1.append(i_q1)
    age_q3.append(i_q3)
    age_iqr.append(i_q3 - i_q1)

# 나이를 인덱스로 하는 DataFrame으로 변환하고 인덱스 이름을 'age'로 설정
age_iqr_df = pd.DataFrame({
    'Q1': age_q1,
    'Q3': age_q3,
    'IQR': age_iqr
}, index=range(5, 19))

age_iqr_df.index.name = 'Age'


# 나이가 19-22세인 사람들의 Q1, Q3, IQR 계산
age_19_22_q1 = df_test.loc[df_test['Basic_Demos-Age'].between(19, 22), 'Physical-Height(cm)'].quantile(0.25)
age_19_22_q3 = df_test.loc[df_test['Basic_Demos-Age'].between(19, 22), 'Physical-Height(cm)'].quantile(0.75)
age_19_22_iqr = age_19_22_q3 - age_19_22_q1

# 나이대 19-22세 행을 추가한 새로운 데이터프레임 생성
age_iqr_df.loc['19-22'] = [age_19_22_q1, age_19_22_q3, age_19_22_iqr]


for age in range(5, 19):
    q1 = age_iqr_df.loc[age, 'Q1']
    q3 = age_iqr_df.loc[age, 'Q3']
    iqr = age_iqr_df.loc[age, 'IQR']
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Applying the outlier bounds to replace values outside of this range with NaN
    df_test.loc[(df_test['Basic_Demos-Age'] == age) & 
                 ((df_test['Physical-Height(cm)'] < lower_bound) | (df_test['Physical-Height(cm)'] > upper_bound)), 
                 'Physical-Height(cm)'] = None
    
q1_19_22 = age_iqr_df.loc['19-22', 'Q1']
q3_19_22 = age_iqr_df.loc['19-22', 'Q3']
iqr_19_22 = age_iqr_df.loc['19-22', 'IQR']
lower_bound_19_22 = q1_19_22 - 1.5 * iqr_19_22
upper_bound_19_22 = q3_19_22 + 1.5 * iqr_19_22

df_test.loc[(df_test['Basic_Demos-Age'].between(19, 22)) & 
             ((df_test['Physical-Height(cm)'] < lower_bound_19_22) | (df_test['Physical-Height(cm)'] > upper_bound_19_22)), 
             'Physical-Height(cm)'] = None


# 'Physical-BMI' 이상치 제거 
df_test.loc[(df_test['Physical-BMI'].notnull()) & 
             (df_test['Physical-Weight(kg)'].isnull() | df_test['Physical-Height(cm)'].isnull()), 
             'Physical-BMI'] = None

df_test['Physical-BMI(calc)'] = df_test['Physical-Weight(kg)'] / (df_test['Physical-Height(cm)'] / 100) ** 2
df_test['BMI_diff'] = abs(df_test['Physical-BMI(calc)'] - df_test['Physical-BMI'])

#단위변환한 변수가 있는 변수들은 제거
df_test.drop(['Physical-BMI', 'BMI_diff'], axis = 1, inplace = True, errors = 'ignore')


# Physical-Waist_Circumference 이상치 제거
age_q1 = []
age_q3 = []
age_iqr = []

for i in range(5, 19):
    # 각 나이에 대한 1사분위수(Q1)와 3사분위수(Q3) 계산
    i_q1 = df_test.loc[df_test['Basic_Demos-Age'] == i, 'Physical-Waist_Circumference'].quantile(0.25)
    i_q3 = df_test.loc[df_test['Basic_Demos-Age'] == i, 'Physical-Waist_Circumference'].quantile(0.75)
    
    # Q1, Q3, IQR 값을 각각의 리스트에 추가
    age_q1.append(i_q1)
    age_q3.append(i_q3)
    age_iqr.append(i_q3 - i_q1)

# 나이를 인덱스로 하는 DataFrame으로 변환하고 인덱스 이름을 'age'로 설정
age_iqr_df = pd.DataFrame({
    'Q1': age_q1,
    'Q3': age_q3,
    'IQR': age_iqr
}, index=range(5, 19))

age_iqr_df.index.name = 'Age'



# 나이가 19-22세인 사람들의 Q1, Q3, IQR 계산
age_19_22_q1 = df_test.loc[df_test['Basic_Demos-Age'].between(19, 22), 'Physical-Waist_Circumference'].quantile(0.25)
age_19_22_q3 = df_test.loc[df_test['Basic_Demos-Age'].between(19, 22), 'Physical-Waist_Circumference'].quantile(0.75)
age_19_22_iqr = age_19_22_q3 - age_19_22_q1

# 나이대 19-22세 행을 추가한 새로운 데이터프레임 생성
age_iqr_df.loc['19-22'] = [age_19_22_q1, age_19_22_q3, age_19_22_iqr]


for age in range(5, 19):
    q1 = age_iqr_df.loc[age, 'Q1']
    q3 = age_iqr_df.loc[age, 'Q3']
    iqr = age_iqr_df.loc[age, 'IQR']
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Applying the outlier bounds to replace values outside of this range with NaN
    df_test.loc[(df_test['Basic_Demos-Age'] == age) & 
                 ((df_test['Physical-Waist_Circumference'] < lower_bound) | (df_test['Physical-Waist_Circumference'] > upper_bound)), 
                 'Physical-Waist_Circumference'] = None
    
q1_19_22 = age_iqr_df.loc['19-22', 'Q1']
q3_19_22 = age_iqr_df.loc['19-22', 'Q3']
iqr_19_22 = age_iqr_df.loc['19-22', 'IQR']
lower_bound_19_22 = q1_19_22 - 1.5 * iqr_19_22
upper_bound_19_22 = q3_19_22 + 1.5 * iqr_19_22

df_test.loc[(df_test['Basic_Demos-Age'].between(19, 22)) & 
             ((df_test['Physical-Waist_Circumference'] < lower_bound_19_22) | (df_test['Physical-Waist_Circumference'] > upper_bound_19_22)), 
             'Physical-Waist_Circumference'] = None

# Physical-HeartRate 이상치 제거
age_band_q1 = []
age_band_q3 = []
age_band_iqr = []

for i in range(0, 3):
    i_q1 = df_test.loc[df_test['Age_band'] == i, 'Physical-HeartRate'].quantile(0.25)
    i_q3 = df_test.loc[df_test['Age_band'] == i, 'Physical-HeartRate'].quantile(0.75)
    
    # Q1, Q3, IQR 값을 각각의 리스트에 추가
    age_band_q1.append(i_q1)
    age_band_q3.append(i_q3)
    age_band_iqr.append(i_q3 - i_q1)

# 나이를 인덱스로 하는 DataFrame으로 변환하고 인덱스 이름을 'age'로 설정
age_band_iqr = pd.DataFrame({
    'Q1': age_band_q1,
    'Q3': age_band_q3,
    'IQR': age_band_iqr
}, index=range(0, 3))

age_band_iqr.index.name = 'Age_band'


for age_band in range(0, 3):
    q1 = age_band_iqr.loc[age_band, 'Q1']
    q3 = age_band_iqr.loc[age_band, 'Q3']
    iqr = age_band_iqr.loc[age_band, 'IQR']
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Applying the outlier bounds to replace values outside of this range with NaN
    df_test.loc[(df_test['Age_band'] == age_band) & 
                 ((df_test['Physical-HeartRate'] < lower_bound) | (df_test['Physical-HeartRate'] > upper_bound)), 
                 'Physical-HeartRate'] = None

# Physical-Systolic_BP, Physical-Diastolic_BP 이상치 제거
rows_to_swap = df_test['Physical-Systolic_BP'] < df_test['Physical-Diastolic_BP']
df_test.loc[rows_to_swap, ['Physical-Systolic_BP', 'Physical-Diastolic_BP']]


# 수축기 혈압이 이완기 혈압보다 낮은 Case는 3가지. 이는 말이 안되므로 3가지 case는 수치 변경함

# In[53]:


rows_to_swap = df_test['Physical-Systolic_BP'] < df_test['Physical-Diastolic_BP']
df_test.loc[rows_to_swap, ['Physical-Systolic_BP', 'Physical-Diastolic_BP']] = df_test.loc[rows_to_swap, ['Physical-Diastolic_BP', 'Physical-Systolic_BP']].values


# ## Age_band 별로 IQR 구하고 이상치 제거 - 'Physical-Systolic_BP'
df_test.groupby('Age_band')['Physical-Systolic_BP'].mean()


age_band_q1 = []
age_band_q3 = []
age_band_iqr = []

for i in range(0, 3):
    i_q1 = df_test.loc[df_test['Age_band'] == i, 'Physical-Systolic_BP'].quantile(0.25)
    i_q3 = df_test.loc[df_test['Age_band'] == i, 'Physical-Systolic_BP'].quantile(0.75)
    
    # Q1, Q3, IQR 값을 각각의 리스트에 추가
    age_band_q1.append(i_q1)
    age_band_q3.append(i_q3)
    age_band_iqr.append(i_q3 - i_q1)

# 나이를 인덱스로 하는 DataFrame으로 변환하고 인덱스 이름을 'age'로 설정
age_band_iqr = pd.DataFrame({
    'Q1': age_band_q1,
    'Q3': age_band_q3,
    'IQR': age_band_iqr
}, index=range(0, 3))

age_band_iqr.index.name = 'Age_band'


for age_band in range(0, 3):
    q1 = age_band_iqr.loc[age_band, 'Q1']
    q3 = age_band_iqr.loc[age_band, 'Q3']
    iqr = age_band_iqr.loc[age_band, 'IQR']
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Applying the outlier bounds to replace values outside of this range with NaN
    df_test.loc[(df_test['Age_band'] == age_band) & 
                 ((df_test['Physical-Systolic_BP'] < lower_bound) | (df_test['Physical-Systolic_BP'] > upper_bound)), 
                 'Physical-Systolic_BP'] = None




# ## Age_band 별로 IQR 구하고 이상치 제거 - 'Physical-Diastolic_BP'

age_band_q1 = []
age_band_q3 = []
age_band_iqr = []

for i in range(0, 3):
    i_q1 = df_test.loc[df_test['Age_band'] == i, 'Physical-Diastolic_BP'].quantile(0.25)
    i_q3 = df_test.loc[df_test['Age_band'] == i, 'Physical-Diastolic_BP'].quantile(0.75)
    
    # Q1, Q3, IQR 값을 각각의 리스트에 추가
    age_band_q1.append(i_q1)
    age_band_q3.append(i_q3)
    age_band_iqr.append(i_q3 - i_q1)

# 나이를 인덱스로 하는 DataFrame으로 변환하고 인덱스 이름을 'age'로 설정
age_band_iqr = pd.DataFrame({
    'Q1': age_band_q1,
    'Q3': age_band_q3,
    'IQR': age_band_iqr
}, index=range(0, 3))

age_band_iqr.index.name = 'Age_band'

for age_band in range(0, 3):
    q1 = age_band_iqr.loc[age_band, 'Q1']
    q3 = age_band_iqr.loc[age_band, 'Q3']
    iqr = age_band_iqr.loc[age_band, 'IQR']
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Applying the outlier bounds to replace values outside of this range with NaN
    df_test.loc[(df_test['Age_band'] == age_band) & 
                 ((df_test['Physical-Diastolic_BP'] < lower_bound) | (df_test['Physical-Diastolic_BP'] > upper_bound)), 
                 'Physical-Diastolic_BP'] = None


# FGC Data
fgc_columns = [col for col in df_test.columns if 'FGC-FGC' in col]
df_test[fgc_columns].describe()



# Initialize lists to store Q1, Q3, and IQR values for each age_band for all FGC columns
fgc_stats = {col: {'Q1': [], 'Q3': [], 'IQR': []} for col in fgc_columns}

# Calculate Q1, Q3, and IQR for each age_band for each FGC column
for col in fgc_columns:
    for i in range(0, 3):  # Assuming Age_band ranges from 0 to 2
        q1 = df_test.loc[df_test['Age_band'] == i, col].quantile(0.25)
        q3 = df_test.loc[df_test['Age_band'] == i, col].quantile(0.75)
        iqr = q3 - q1
        
        # Append the calculated values to the corresponding lists
        fgc_stats[col]['Q1'].append(q1)
        fgc_stats[col]['Q3'].append(q3)
        fgc_stats[col]['IQR'].append(iqr)

# Apply outlier bounds and replace outliers with NaN
for col in fgc_columns:
    for age_band in range(0, 3):
        q1 = fgc_stats[col]['Q1'][age_band]
        q3 = fgc_stats[col]['Q3'][age_band]
        iqr = fgc_stats[col]['IQR'][age_band]
        lower_bound = q1 - 3 * iqr
        upper_bound = q3 + 3 * iqr

        # Replace values outside the bounds with NaN
        df_test.loc[(df_test['Age_band'] == age_band) & 
                     ((df_test[col] < lower_bound) | (df_test[col] > upper_bound)), col] = None


# BIA Data
bia_columns = [col for col in df_test.columns if col.startswith('BIA-BIA')]

# 나이별로 각 'BIA-BIA' 열의 Q1, Q3, IQR을 계산하고 이상치를 결측치로 변환
for column in bia_columns:
    # 나이별로 Q1, Q3, IQR을 저장할 데이터프레임 생성
    age_q1 = []
    age_q3 = []
    age_iqr = []
    
    for age in range(5, 19):
        i_q1 = df_test.loc[df_test['Basic_Demos-Age'] == age, column].quantile(0.25)
        i_q3 = df_test.loc[df_test['Basic_Demos-Age'] == age, column].quantile(0.75)
        age_q1.append(i_q1)
        age_q3.append(i_q3)
        age_iqr.append(i_q3 - i_q1)
    
    # 데이터프레임 생성하여 나이별 Q1, Q3, IQR 값 저장
    age_iqr_df = pd.DataFrame({
        'Q1': age_q1,
        'Q3': age_q3,
        'IQR': age_iqr
    }, index=range(5, 19))
    
    # 19-22세 나이 그룹에 대한 Q1, Q3, IQR 계산 및 데이터프레임에 추가
    age_19_22_q1 = df_test.loc[df_test['Basic_Demos-Age'].between(19, 22), column].quantile(0.25)
    age_19_22_q3 = df_test.loc[df_test['Basic_Demos-Age'].between(19, 22), column].quantile(0.75)
    age_19_22_iqr = age_19_22_q3 - age_19_22_q1
    age_iqr_df.loc['19-22'] = [age_19_22_q1, age_19_22_q3, age_19_22_iqr]
    
    # 각 나이대에 대해 IQR 기준으로 이상치를 결측치로 대체
    for age in range(5, 19):
        q1 = age_iqr_df.loc[age, 'Q1']
        q3 = age_iqr_df.loc[age, 'Q3']
        iqr = age_iqr_df.loc[age, 'IQR']
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        
        # 이상치 범위를 벗어난 값들을 None으로 대체
        df_test.loc[(df_test['Basic_Demos-Age'] == age) & 
                     ((df_test[column] < lower_bound) | (df_test[column] > upper_bound)), column] = None
    
    # 19-22세 그룹 이상치 처리
    lower_bound_19_22 = age_19_22_q1 - 1.5 * age_19_22_iqr
    upper_bound_19_22 = age_19_22_q3 + 1.5 * age_19_22_iqr
    
    df_test.loc[(df_test['Basic_Demos-Age'].between(19, 22)) & 
                 ((df_test[column] < lower_bound_19_22) | (df_test[column] > upper_bound_19_22)), column] = None



In [18]:
# test KNN_sequential

numeric_cols = df_test.select_dtypes(include=['int64', 'float64']).columns

# Step 3: Separate numeric and non-numeric columns
non_numeric_cols = df_test.select_dtypes(exclude=['int64', 'float64']).columns
numeric_data = df_test[numeric_cols]
non_numeric_data = df_test[non_numeric_cols]

# Step 4: Calculate missing values ratio for numeric columns
missing_ratios = numeric_data.isnull().mean().sort_values()

# Ensure 'PreInt_EduHx-computerinternet_hoursday' is imputed last
if 'PreInt_EduHx-computerinternet_hoursday' in missing_ratios.index:
    missing_ratios = missing_ratios.drop('PreInt_EduHx-computerinternet_hoursday')
    missing_ratios['PreInt_EduHx-computerinternet_hoursday'] = 1.0  # Set it as the last column to impute

# Step 5: Sequentially impute missing values using KNN Imputer
imputer = KNNImputer(n_neighbors=4)
numeric_data_imputed = numeric_data.copy()

for col in tqdm(missing_ratios.index, desc="Columns Progress"):
    if numeric_data[col].isnull().sum() > 0:
        # Impute the target column using KNN Imputer
        numeric_data_imputed[col] = imputer.fit_transform(numeric_data_imputed)[
            :, numeric_data.columns.get_loc(col)
        ]

# Step 6: Combine imputed numeric data with non-numeric data
df_imputed = pd.concat([numeric_data_imputed, non_numeric_data], axis=1)

# Step 7: Convert integer-like float columns back to int where necessary
for col in numeric_data_imputed.select_dtypes(include=['float64']).columns:
    if col in df_test.columns and df_test[col].dtype == 'int64':
        df_imputed[col] = df_imputed[col].round().astype(int)

# Step 8: Round float columns to 2 decimal places
for col in numeric_data_imputed.select_dtypes(include=['float64']).columns:
    if col in df_test.columns and df_test[col].dtype == 'float64':
        df_imputed[col] = df_imputed[col].round(2)

# Step 9: Verify that all missing values have been filled
print(df_imputed.isnull().sum())

df_test = df_imputed
test_imputed = df_imputed

Columns Progress: 100%|██████████| 33/33 [00:00<00:00, 132.33it/s]

Basic_Demos-Age                           0
Basic_Demos-Sex                           0
CGAS-CGAS_Score                           0
Physical-Waist_Circumference              0
Physical-Diastolic_BP                     0
Physical-HeartRate                        0
Physical-Systolic_BP                      0
FGC-FGC_CU                                0
FGC-FGC_PU                                0
FGC-FGC_SRL                               0
FGC-FGC_SRR                               0
FGC-FGC_TL                                0
BIA-BIA_Activity_Level_num                0
BIA-BIA_BMC                               0
BIA-BIA_BMI                               0
BIA-BIA_BMR                               0
BIA-BIA_DEE                               0
BIA-BIA_ECW                               0
BIA-BIA_FFM                               0
BIA-BIA_FFMI                              0
BIA-BIA_FMI                               0
BIA-BIA_Fat                               0
BIA-BIA_Frame_num               

In [19]:
# test_PCA

#단위변환한 변수가 있는 변수들은 제거
test_imputed.drop(['Age_Group','Physical-BMI', 'Physical-Height','Physical-Weight', 'BIA-BIA_BMI'], axis = 1, inplace = True, errors = 'ignore')

# 'id' 열을 제외한 상태로 처리
id_column = test_imputed['id']  # 'id' 열 따로 저장

# Label encode categorical columns (excluding 'id')
categorical_cols = test_imputed.select_dtypes(include=['object']).columns.difference(['id'])
label_encoder = LabelEncoder()


# 선택한 특성 추출
test_imputed['FGC-FGC_SR'] = (test_imputed['FGC-FGC_SRL'] + test_imputed['FGC-FGC_SRR'])/2
FGC_SR = test_imputed[['FGC-FGC_SRL', 'FGC-FGC_SRR']]

# 데이터 표준화
scaler = StandardScaler()
features_scaled = scaler.fit_transform(FGC_SR)

# PCA 모델 설정, 주성분 개수를 1로 설정
pca = PCA(n_components=1)

# PCA 적용
FGC_SR = pca.fit_transform(FGC_SR)

# 설명된 분산 비율 확인
explained_variance = pca.explained_variance_ratio_

#Sit_Reach 주성분을 데이터 프레임으로 만들기
FGC_SR_df = pd.DataFrame(data = FGC_SR, columns = ['FGC_SR'])




# 선택한 특성 추출
BIA_features = test_imputed[['BIA-BIA_BMC','BIA-BIA_FMI','BIA-BIA_BMR','BIA-BIA_DEE', 'BIA-BIA_ECW','BIA-BIA_FFM','BIA-BIA_FFMI','BIA-BIA_Fat','BIA-BIA_ICW','BIA-BIA_LDM','BIA-BIA_LST','BIA-BIA_SMM','BIA-BIA_TBW']]

# 데이터 표준화
scaler = StandardScaler()
features_scaled = scaler.fit_transform(BIA_features)

# PCA 모델 설정
pca = PCA(n_components=1)

# PCA 적용
BIA = pca.fit_transform(BIA_features)

# 설명된 분산 비율 확인
explained_variance = pca.explained_variance_ratio_

#BIA 주성분을 데이터 프레임으로 만들기
BIA_df = pd.DataFrame(data = BIA, columns = ['BIA'])

# Ensure 'id' column is intact in the final concatenated DataFrame
test_concat = pd.concat([test_imputed, FGC_SR_df, BIA_df], axis=1)

#PCA를 통해 축소한 feature들을 제거
test_concat.drop(['BMI_diff','FGC-FGC_SR', 'FGC-FGC_SRL','FGC-FGC_SRR','BIA-BIA_BMC', 'BIA-BIA_BMR','BIA-BIA_DEE', 'BIA-BIA_ECW','BIA-BIA_FFM','BIA-BIA_FFMI','BIA-BIA_Fat','BIA-BIA_ICW','BIA-BIA_LDM','BIA-BIA_LST','BIA-BIA_SMM','BIA-BIA_TBW','BIA-BIA_FMI'], axis =1, inplace = True, errors = 'ignore')

# #PCIAT_Total Column을 추가하는 코드
# df_pciat_total = pd.read_csv('test.csv')
# test_concat1 = pd.concat([test_concat, df_pciat_total], axis = 1)

#처리한 df를 df_test이름으로 변경
df_test = test_concat

In [20]:
df_test_origin = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')

# 'id' 열만 추출
id_column = df_test_origin['id']

# 기존 df_test 데이터프레임에 'id' 열 추가
df_test['id'] = id_column

In [21]:
df_test.head()

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,FGC-FGC_CU,FGC-FGC_PU,FGC-FGC_TL,...,BIA-BIA_Frame_num,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,Physical-Weight(kg),Physical-Height(cm),Physical-BMI(calc),id,Age_band,FGC_SR,BIA
0,5,0,51.00,25.75,68.25,75.50,108.25,0.00,0.0,6.0,...,1.0,56.50,3.0,23.04,116.84,16.88,00008ff9,0,2.012836,-544.446247
1,9,0,64.25,22.00,75.00,70.00,122.00,3.00,5.0,3.0,...,1.0,64.00,0.0,20.87,121.92,14.04,000fd460,0,-4.378359,-537.287710
2,10,1,71.00,26.25,65.00,94.00,117.00,20.00,7.0,5.0,...,2.0,54.00,2.0,38.10,143.51,18.68,00105258,0,-2.965885,159.191342
3,9,0,71.00,26.25,60.00,97.00,117.00,18.00,5.0,7.0,...,2.0,45.00,0.0,37.01,142.24,18.29,00115b9f,0,1.271537,-77.248821
4,18,1,60.75,26.25,60.50,86.25,109.75,12.75,5.0,9.5,...,2.0,48.75,2.0,40.60,146.37,18.95,0016bb22,1,-1.756266,159.191342


In [22]:
# 'Age_band' 열을 숫자형으로 변환 (간단히 처리)
if 'Age_band' in df_test.columns:
    df_test['Age_band'] = df_test['Age_band'].astype(str).astype(float)

# 학습에 사용한 feature만 선택
train_features = X.columns.tolist()  # 학습에 사용한 feature 이름 리스트

# 테스트 데이터에서 동일한 feature만 선택
df_test_features = df_test[train_features]

# 예측 수행
df_test['Predicted_PCIAT'] = final_model.predict(df_test_features)

# SII 값 변환
df_test['sii'] = df_test['Predicted_PCIAT'].apply(determine_sii)

# id와 sii 열만 선택
result_df = df_test[['id', 'sii']]

# 결과 저장
result_df.to_csv('submission.csv', index=False)

print(result_df)

          id  sii
0   00008ff9    1
1   000fd460    0
2   00105258    1
3   00115b9f    0
4   0016bb22    1
5   001f3379    1
6   0038ba98    1
7   0068a485    0
8   0069fbed    1
9   0083e397    1
10  0087dd65    0
11  00abe655    0
12  00ae59c9    1
13  00af6387    1
14  00bd4359    1
15  00c0cd71    0
16  00d56d4b    0
17  00d9913d    1
18  00e6167c    0
19  00ebc35d    1
